# Collaborative Filtering Book Recommender System

In [3]:
# import dependencies
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

/var/folders/b6/wvsnhxp56z97rld_stlsjcrh0000gn/T/ipykernel_91158/4048939954.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [6]:
# read datasets
books_df = pd.read_csv('/Users/reddragon/Documents/SMU/Y2S2/IS215 Digital Business/project /book_recommendation_dataset/Books.csv', nrows=2000)
users_df = pd.read_csv('/Users/reddragon/Documents/SMU/Y2S2/IS215 Digital Business/project /book_recommendation_dataset/Users.csv')
ratings_df = pd.read_csv('/Users/reddragon/Documents/SMU/Y2S2/IS215 Digital Business/project /book_recommendation_dataset/Ratings.csv')

In [7]:
books_df.head()
# users_df.head()
# ratings_df.head()
books_df.shape

(2000, 8)

In [8]:
user_ratings_df = ratings_df.merge(users_df, left_on = 'User-ID', right_on = 'User-ID')
user_ratings_df.head()

,User-ID,ISBN,Book-Rating,Location,Age
0,276725,034545104X,0,"tyler, texas, usa",NaN
1,276726,0155061224,5,"seattle, washington, usa",NaN
2,276727,0446520802,0,"h, new south wales, australia",16.0
3,276729,052165615X,3,"rijeka, n/a, croatia",16.0
4,276729,0521795028,6,"rijeka, n/a, croatia",16.0


<h4> Collaborative Filtering </h4>

In [9]:
book_user_rating = books_df.merge(user_ratings_df, left_on = 'ISBN',right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace = True)

In [10]:
d ={}
for i,j in enumerate(book_user_rating.ISBN.unique()):
    d[j] =i
book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)

In [11]:
users_books_pivot_matrix_df = book_user_rating.pivot(index='User-ID', 
                                                          columns='unique_id_book', 
                                                          values='Book-Rating').fillna(0)

In [12]:
users_books_pivot_matrix_df.head()

unique_id_book,0,1,2,3,4,5,6,7,8,9,...,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995
User-ID,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
users_books_pivot_matrix_df = users_books_pivot_matrix_df.values
users_books_pivot_matrix_df

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
from scipy.sparse.linalg import svds

NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_books_pivot_matrix_df, k = NUMBER_OF_FACTORS_MF)

In [15]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [16]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[-2.68644574e-30,  4.15832091e-16, -1.69927296e-19, ...,
        -4.87234191e-17,  5.42104563e-16,  5.39446396e-16],
       [-2.64125314e-18,  8.65244939e-03, -1.21394642e-06, ...,
         8.09023897e-04,  8.95013879e-03,  1.04237422e-02],
       [-1.31607356e-17,  1.27588381e-02,  2.23224187e-06, ...,
         8.60297636e-04,  1.51855387e-02,  1.61201717e-02],
       ...,
       [-3.99333769e-17,  1.78452255e-02,  3.36923115e-06, ...,
         2.18327370e-03,  1.80608432e-02,  2.31052785e-02],
       [-3.45420693e-17,  2.15177143e-02,  1.34648118e-05, ...,
         1.02930707e-03,  2.72015318e-02,  3.41269336e-02],
       [-1.87670465e-17,  2.71719603e-02,  1.53034940e-06, ...,
         2.08854266e-03,  2.79393000e-02,  3.39456989e-02]])

<h4> Functions </h4>

In [17]:
def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id 
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        print(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])

<h4> Output </h4>

In [20]:
k = 50
movie_id =1234  
top_n = 3
sliced = Vt.T[:, :k] # representative data

similar_books(book_user_rating, movie_id, top_cosine_similarity(sliced, movie_id, top_n))

Recommendations for Der Fanger Im Roggen: 

Demian. Die Geschichte von Emil Sinclairs Jugend.
All of Me: A Voluptuous Tale
Watership Down
